<a href="https://colab.research.google.com/github/trian-ctrn/envitranslate/blob/main/entovitrans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
!pip install gradio

In [ ]:
!pip install vncorenlp

In [ ]:
import gradio as gr
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM , pipeline
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
import sentencepiece
import time
import torch
from vncorenlp import VnCoreNLP
from nltk.corpus import wordnet
import gensim
from nltk.data import find
nltk.download('word2vec_sample')

In [ ]:
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [ ]:
!git clone  https://github.com/vncorenlp/VnCoreNLP


In [ ]:
annotator = VnCoreNLP("/content/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg,pos")

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos")

model_pos = AutoModelForTokenClassification.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos")  


In [ ]:
def vietnamese_postag(input):
  annotated_text = annotator.annotate(input)
  target_dict = []
  for first_dict in annotated_text.values():
    for another_dict in first_dict:
      for another_another_dict in another_dict:
        res = {key: another_another_dict[key] for key in another_another_dict.keys()
                               & {'form','posTag,ner'}} 
        target_dict.append(res)

  split_text = []
  pos_tag = []
  text = []
  for i in range(len(target_dict)):
    split_text.append(target_dict[i]['form'])
    pos_tag.append(target_dict[i]['posTag'])
  for i in range(len(split_text)):
    text.extend([(split_text[i],pos_tag[i]),(" ",None)])
  return text

In [ ]:
model_path_en2vi = "Yama/yamavi"
model_path_vi2en = "Yama/yamaen"
def load_model(model_path_en2vi,model_path_vi2en):
    tokenizer_en2vi = AutoTokenizer.from_pretrained(model_path_en2vi)
    model_en2vi = AutoModelForSeq2SeqLM.from_pretrained(model_path_en2vi)
    tokenizer_vi2en = AutoTokenizer.from_pretrained(model_path_vi2en)
    model_vi2en = AutoModelForSeq2SeqLM.from_pretrained(model_path_vi2en)      
    return model_en2vi, tokenizer_en2vi,model_vi2en,tokenizer_vi2en
model_entovi , tokenizer_entovi, model_vitoen, tokenizer_vitoen  = load_model(model_path_en2vi,model_path_vi2en) 

In [ ]:
label = {
    "0": "ADJ",
    "1": "ADP",
    "2": "ADV",
    "3": "AUX",
    "4": "CCONJ",
    "5": "DET",
    "6": "INTJ",
    "7": "NOUN",
    "8": "NUM",
    "9": "PART",
    "10": "PRON",
    "11": "PROPN",
    "12": "PUNCT",
    "13": "SCONJ",
    "14": "SYM",
    "15": "VERB",
    "16": "X"
  }


In [ ]:

def entovi(input):  
    tokenized_txt = tokenizer_entovi(input,return_tensors = 'pt').input_ids
    outputs = model_entovi.generate(tokenized_txt)
    return tokenizer_entovi.decode(outputs[0], skip_special_tokens = True)
def vitoen(input):  
    tokenized_txt = tokenizer_entovi(input,return_tensors = 'pt').input_ids
    outputs = model_vitoen.generate(tokenized_txt)
    return tokenizer_vitoen.decode(outputs[0], skip_special_tokens = True)
def english_postag(input):
  label_id = []
  split_text = input.split(' ')
  inputs = tokenizer(input, add_special_tokens=True, return_tensors="pt")
  with torch.no_grad():
    logits = model_pos(**inputs).logits

  predicted_token_class_ids = logits.argmax(-1)
  for i in predicted_token_class_ids[0]:
      label_id.append(label[str(i.item())])
  del(label_id[0])
  del(label_id[-1])
  text = []
  for i in range(len(split_text)):
    text.extend([(split_text[i], label_id[i]),(" ",None)])
  return text
def vietnamese_postag(input):
  annotated_text = annotator.annotate(input)
  target_dict = []
  for first_dict in annotated_text.values():
    for another_dict in first_dict:
      for another_another_dict in another_dict:
        res = {key: another_another_dict[key] for key in another_another_dict.keys()
                               & {'form','posTag'}} 
        target_dict.append(res)

  split_text = []
  pos_tag = []
  text = []
  for i in range(len(target_dict)):
    split_text.append(target_dict[i]['form'])
    pos_tag.append(target_dict[i]['posTag'])
  for i in range(len(split_text)):
    text.extend([(split_text[i],pos_tag[i]),(" ",None)])
  return text
def greet(Choose,input):#,audio,state=""):
    if Choose == "Eng to Vi":
      return vietnamese_postag(entovi(input)) 
    if Choose == "Vi to Eng":
      return english_postag(vitoen(input)) 
def w2em(list_input):
  synonyms = model.most_similar(positive=list_input.split(' '), topn = 10)
  words = []
  for i in range(10):
    words.append(synonyms[i][0])
  return ", ".join(words)

In [ ]:
demo = gr.Interface(fn = w2em, 
                         inputs = [gr.Textbox(placeholder="Example: King Woman Royal")],
                                            #  "number"],
                        outputs = [gr.Textbox()]
                     )


In [ ]:
iface = gr.Interface(fn = greet,
                     inputs = [gr.inputs.Dropdown(["Eng to Vi", "Vi to Eng"]),
                               gr.Textbox(placeholder="Enter sentence here...")],
                               #gr.inputs.Audio(source="microphone",type="filepath")],
                     outputs=["highlight"],
                     description = "Write a full short sentence(maximum length = 80), shouldn't write a word"
                     )


In [ ]:
hola = gr.TabbedInterface([demo,iface], ["Advanced Search", "Translate"])
hola.launch()